# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')

In [2]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
click_list = list(df.loc[df.action == 'click']['id'])

In [4]:
view_list = list(df.loc[df.action == 'view']['id'])

In [5]:
len(click_list)

1860

In [6]:
len(view_list)

6328

In [7]:
for id in click_list:
    if id not in view_list:
        print(id)
#no anomaly    

In [8]:
# of click is 1860

In [9]:
cid = df[df.action=='click']['id'].unique()
vids = df[df.action=='view']['id'].unique()

In [10]:
print(len(cid))
print(len(vids))

1860
6328


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [12]:
df.id.unique()

array([804196, 434745, 507599, ..., 458115, 505451, 461199])

In [13]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [14]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [15]:
e_group = set(df[df.group=='experiment']['id'].unique())
c_group = set(df[df.group=='control']['id'].unique())
print(len(e_group & c_group))

0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [16]:
#Your code here
# compare mean click ratio

e_group = df[df.group=='experiment']
print(len(e_group))
e_group_click = df[(df.group=='experiment') & (df.action=='click')]
print(len(e_group_click))

3924
928


In [17]:
click_ratio_experiment = len(e_group_click) / len(e_group)
print(click_ratio_experiment)

0.23649337410805302


In [20]:
c_group = df[df.group=='control']
print(len(c_group))
c_group_click = df[(df.group=='control') & (df.action=='click')]
print(len(c_group_click))

4264
932


In [23]:
4264-932

3332

In [22]:
click_ratio_control = len(c_group_click) / len(c_group)
click_ratio_control

0.21857410881801126

In [24]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [28]:
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')

In [29]:
print(len(control))
print(len(experiment))

3332
2996


In [32]:
control

action,click,view
id,,
182994,1.0,1.0
183089,NaN,1.0
183248,1.0,1.0
183515,NaN,1.0
183524,NaN,1.0
...,...,...
936786,NaN,1.0
937003,NaN,1.0
937073,NaN,1.0


In [34]:
print(control.click.sum())

932.0


In [33]:
print(experiment.click.sum())

928.0


In [38]:
click_through_rate_experiment = experiment.click.sum() / len(experiment)
print(click_through_rate_experiment)

0.3097463284379172


In [37]:
click_through_rate_control = control.click.sum() / len(control)
print(click_through_rate_control)

0.2797118847539016


In [52]:
from scipy import stats
stats.ttest_ind(control.click, experiment.click)

Ttest_indResult(statistic=nan, pvalue=nan)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [40]:
control.click.mean()

1.0

In [44]:
#Your code here
expected_click = len(experiment) * click_through_rate_control

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [43]:
#Your code here
import numpy as np
n = len(experiment)
p = click_through_rate_control
var = n * p * (1-p)
std = np.sqrt(var)
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [47]:
z_score = (experiment.click.sum() - expected_click) / std

In [48]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z_score)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.